### En este notebook se debe de utilizar los archivos data_loader y processor para realizar la comparación y la estimación del número de entradas a la estación chamartín. 

In [1]:
import numpy as np
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt
from processor2 import * #Importamos todas las clases del archivo processor

In [2]:
# #Verificamos con el df timeseries_o
timeseries_o = pd.read_csv("/home/jonathan/tesis/AppDaniel/TFG_CODE2/app/data/trips_data/timeseries_o.csv", sep=',',index_col=0, parse_dates=True)
timeseries_o = timeseries_o.loc['2020-06-21':'2020-07-31 23:00:00']
timeseries_o_2 = timeseries_o.copy()
# columnas_o = [columna for columna in Period_Nn.columns if columna.startswith('2807905')]
# Period_Nn = Period_Nn[columnas_o]
# Period_Nn.tail(25)

# chamartin_o para el periodo de nueva normalidad: 21 Jun - 31 Jul 2020

In [3]:
Period_Nn = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento2.csv", sep=',',index_col=0, parse_dates=True)
Period_Nn = Period_Nn.set_index('ds')
Period_Nn = Period_Nn.loc['2020-06-21':'2020-07-31 23:00:00']
columnas_o = [columna for columna in Period_Nn.columns if columna.startswith('2807905')]
subsetNn = Period_Nn[columnas_o]
subsetNn2 = subsetNn.copy()
# Realizamos la suma del distrito origen chamartín
subsetNn['2807905'] = subsetNn.loc[:, columnas_o].sum(axis=1)
colum_sum = subsetNn.loc[:, '2807905']
index = subsetNn.index
chamartin_o = pd.DataFrame({'2807905': colum_sum})
chamartin_o.index = index
# chamartin_o.tail(5)

/tmp/ipykernel_8420/1369727985.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subsetNn['2807905'] = subsetNn.loc[:, columnas_o].sum(axis=1)


# Estimación de las entradas a la estación Chamartín a partir de las predicciones del trayecto "Chamartín-Centro"

## Read - Archivos con las predicciones obtenidas

In [4]:
predictions1_0_5 = pd.read_csv("/home/jonathan/tesis/10mo_avance/Evaluación/predictions1_0_5.csv", sep=',',index_col=0, parse_dates=True)
predictions2_0_5 = pd.read_csv("/home/jonathan/tesis/10mo_avance/Evaluación/predictions2_0_5.csv", sep=',',index_col=0, parse_dates=True)
predictions1_6_11 = pd.read_csv("/home/jonathan/tesis/10mo_avance/Evaluación/predictions1_6_11.csv", sep=',',index_col=0, parse_dates=True)
predictions2_6_11 = pd.read_csv("/home/jonathan/tesis/10mo_avance/Evaluación/predictions2_6_11.csv", sep=',',index_col=0, parse_dates=True)
predictions1_12_17 = pd.read_csv("/home/jonathan/tesis/10mo_avance/Evaluación/predictions1_12_17.csv", sep=',',index_col=0, parse_dates=True)
predictions2_12_17 = pd.read_csv("/home/jonathan/tesis/10mo_avance/Evaluación/predictions2_12_17.csv", sep=',',index_col=0, parse_dates=True)
predictions1_18_23 = pd.read_csv("/home/jonathan/tesis/10mo_avance/Evaluación/predictions1_18_23.csv", sep=',',index_col=0, parse_dates=True)
predictions2_18_23 = pd.read_csv("/home/jonathan/tesis/10mo_avance/Evaluación/predictions2_18_23.csv", sep=',',index_col=0, parse_dates=True)

predictions1 = pd.concat([predictions1_0_5, predictions1_6_11, predictions1_12_17, predictions1_18_23])
predictions2 = pd.concat([predictions2_0_5, predictions2_6_11, predictions2_12_17, predictions2_18_23])

## Añadimos las predicciones en el df "subsetNn2" para obtener el df chamartin_op

In [5]:
valores_reemplazo_20jul = predictions1.loc['2020-07-20', 'yhat'].to_dict()
valores_reemplazo_27jul = predictions2.loc['2020-07-27', 'yhat'].to_dict()

# Reemplaza los registros específicos en la columna '2807905-2807901'
for fecha_hora, valor in valores_reemplazo_20jul.items():
    fecha = fecha_hora.date()
    hora = fecha_hora.time().strftime('%H:%M:%S')       
    subsetNn2.loc[f'{fecha} {hora}', '2807905-2807901'] = valor
    
for fecha_hora, valor in valores_reemplazo_27jul.items():
    fecha = fecha_hora.date()
    hora = fecha_hora.time().strftime('%H:%M:%S')       
    subsetNn2.loc[f'{fecha} {hora}', '2807905-2807901'] = valor

In [6]:
columnas_o = [columna for columna in subsetNn2.columns if columna.startswith('2807905')]
subsetNn2['2807905'] = subsetNn2.loc[:, columnas_o].sum(axis=1)
colum_sum = subsetNn2.loc[:, '2807905']
index = subsetNn2.index
chamartin_op = pd.DataFrame({'2807905': colum_sum})
chamartin_op.index = index

In [7]:
# # Verificamos que se hayan reemplazado los registros por las prediccones
# print(chamartin_o.loc['2020-07-19 23:00:00':'2020-07-20 01:00:00'])
# print(chamartin_op.loc['2020-07-19 23:00:00':'2020-07-20 01:00:00'])
# print(chamartin_o.loc['2020-07-26 23:00:00':'2020-07-27 01:00:00'])
# print(chamartin_op.loc['2020-07-26 23:00:00':'2020-07-27 01:00:00'])

## Añadimos sub1, sub2chamartin_op  en el df "timeseries_o_2" para obtener el df timeseries con las predicciones

In [8]:
sub1chamartin_op = chamartin_op.loc['2020-07-20':'2020-07-20 23:00:00']
sub2chamartin_op = chamartin_op.loc['2020-07-27':'2020-07-27 23:00:00']
# Convertir el index a 'datetime64' para agregar registros de predicciones
sub1chamartin_op.index = pd.to_datetime(sub1chamartin_op.index)
sub2chamartin_op.index = pd.to_datetime(sub2chamartin_op.index)

In [9]:
valores_reemplazo_20jul = sub1chamartin_op.loc['2020-07-20', '2807905'].to_dict()
valores_reemplazo_27jul = sub2chamartin_op.loc['2020-07-27', '2807905'].to_dict()

# Reemplaza los registros específicos en la columna '2807905'
for fecha_hora, valor in valores_reemplazo_20jul.items():
    fecha = fecha_hora.date()
    hora = fecha_hora.time().strftime('%H:%M:%S')       
    timeseries_o_2.loc[f'{fecha} {hora}', '2807905'] = valor
    
for fecha_hora, valor in valores_reemplazo_27jul.items():
    fecha = fecha_hora.date()
    hora = fecha_hora.time().strftime('%H:%M:%S')       
    timeseries_o_2.loc[f'{fecha} {hora}', '2807905'] = valor

# Realizamos la estimación de los usuarios que entran a las estaciones

In [10]:
tripsloader = TripsLoader(verbose = True)
ptdata = PassengersDataLoader()
routeTrip = RouteTrip(tripsloader,ptdata)
renfe_up = routeTrip.get_users_renfe(timeseries_o)
renfe_up2 = routeTrip.get_users_renfe(timeseries_o_2)

/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  beta.up = beta.up / beta.up.sum()
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Us

/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated 

/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated 

/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated 

/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated 

/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  up = up.append(up_aux)
/home/jonathan/tesis/10mo_avance/Evaluación/processor2.py:122: FutureWarning: The frame.append method is deprecated 

# Comparamos los resultados de la estimación con los datos reales en el mes de Julio

In [22]:
renfe_upJUL = renfe_up.loc['2020-07-01':'2020-07-31 23:00:00'] 
sum_por_estacion = renfe_upJUL.sum()
total_usuarios_julio = sum_por_estacion.sum()
print("Número total de usuarios en julio:", total_usuarios_julio)

Número total de usuarios en julio: 10628999.999999998


# Realizamos la evaluación del segmento de interés

In [18]:
Chamartin_up2 = renfe_up['par_5_18']
Chamartin_up2 = Chamartin_up2.loc['2020-07-19 20:00:00':'2020-07-20 06:00:00'] 
Chamartin_up2

# Chamartin_up = renfe_up['par_5_18']
# Chamartin_up = Chamartin_up.loc['2020-07-01':'2020-07-31 23:00:00'] 
# Chamartin_up

ds
2020-07-19 20:00:00    430.515687
2020-07-19 21:00:00    440.245361
2020-07-19 22:00:00    389.895753
2020-07-19 23:00:00    249.751166
2020-07-20 00:00:00    236.906021
2020-07-20 06:00:00    452.050744
Name: par_5_18, dtype: float64